In [ ]:
import dash as dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input,Output
import plotly.express as px
import dash_table
import pandas as pd
import requests


In [ ]:
hakuurl="https://raw.githubusercontent.com/varmais/maakunnat/master/kunnat.geojson"
haku=requests.get(url=hakuurl, timeout=60000)
geojson=haku.json()

In [ ]:
df=pd.read_excel('Sukupuoliikä.xlsx')
df

In [ ]:
df2=pd.read_excel('2000-2019väestöyhteensä.xlsx')
df2 = df2.melt(id_vars=["Alue","Vuosi"], value_vars=["Miehet Väestö 31.12.", "Naiset Väestö 31.12.", "Yhteensä Väestö 31.12."], var_name="sukupuoli", value_name="henkilöt")
df2['Yhteensäerotus'] = df2['henkilöt'].diff()

In [ ]:
df2['Sukupuoli']=df2['sukupuoli']
df2 = df2[df2.Sukupuoli != "Yhteensä Väestö 31.12."]
df2

In [ ]:
asdd=df2['henkilöt']
asddd=asdd.pct_change()
prosentuaalinenerotus = pd.DataFrame(asddd)
prosentuaalinenerotus['pcterotus']=prosentuaalinenerotus['henkilöt']
prosentuaalinenerotus = prosentuaalinenerotus.drop('henkilöt', 1)
prosentuaalinenerotus

In [ ]:
result = pd.concat([df2, prosentuaalinenerotus], axis=1)
result

In [ ]:
asdddd=result.round({'pcterotus': 5})
result['pcterotus']=asdddd['pcterotus'] * 100
result

In [ ]:
df2['pcterotus']=result['pcterotus']
df2

In [ ]:
df3 = df[df.Sukupuoli != "Yhteensä"]
df3=df3[df3.Ikä != "Yhteensä"]
df3

In [ ]:
df4=df2
df4=df4[df4.Vuosi != 2000]

In [ ]:
df4=df4[df4.sukupuoli != "Yhteensä Väestö 31.12."]
df4

In [ ]:
Sukupuoli = df.Sukupuoli.unique()
Ikä = df.Ikä.unique()
Vuosi=df.Vuosi.unique()
Alue=df.Alue.unique()


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
colors = {
    "graphBackground": "#F5F5F5",
    "background": "#ffffff",
    "text": "#123"
}


app.layout=html.Div(children =[
    html.H3("Kalle Karlsson, XAMK"),
    html.H1(
        children='Verkkosivu visualisointeja varten',
        style={
            'textAlign': 'center','color': '#7FDBFF'}),
    
    html.Div([
        html.H4('Hajontakaavio'),
        dcc.Markdown('''
            Tämä visualisaatio esittää kuntien populaation sukupuolittain ja vuosittain
           
            Datan lähde [Stat.fi](https://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__vrm__vaerak/statfin_vaerak_pxt_11re.px/).

        
            ''')]),
    html.Div([
        dcc.Dropdown(id="Ikä", 
                     options=[{"label": x, "value": x} for x in Ikä], 
                     value=Ikä[0],
                     clearable=False),
        
        dcc.Graph(id="graph")],
        
        style={'width': '100%', 'display': 'inline-block'}),
        
    html.Div([    
        html.H1("Choropleth kartta"),
        dcc.Markdown('''
            Tämä visualisaatio esittää naisten ja miesten populaatio määrän muuttumisen prosentteina vuosittain. 
            
            Datan lähde [Stat.fi](https://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__vrm__vaerak/statfin_vaerak_pxt_11re.px/).
            
        
            ''')]),
    html.Div([
        dcc.Graph(id="graph2", figure={}),
       
        dcc.Slider(id='slider',
                   min=df2['Vuosi'][1],
                   max=df2['Vuosi'].max(),
                   value=df2['Vuosi'][1],
                   marks={str(year): str(year) for year in df2['Vuosi'].unique()},
                   step=None)],
        
        style={'width': '100%', 'display': 'inline-block'}),
  
    html.Div([    
        html.H1("Animoitu pylväskaavio sekä aluekaavio"),
        dcc.Markdown('''
            Tämä visualisaatio esittää ikäryhmien muutoksen vuosittain maakunnittain, sekä esittää vuosittaisen muutoksen väestössä sukupuolittain.
        
            ''')]),


    html.Div([
        dcc.Dropdown( id="dropdown",
                     options=[{"label": x, "value": x} for x in Alue],
                     value=Alue[0],
                     clearable=False),
        
            html.Div(children=[
                dcc.Graph(id="graph3", style={'width': '50%', 'display': 'inline-block'}),
                dcc.Graph(id="graph4", style={'width': '50%', 'display': 'inline-block'})])]),
        
        
      html.Div([ 
        html.H1("Sankey diagrammi"),
        dcc.Markdown('''
            Sankey diagrammi esittää maakuntien välisen muuttoliikkeen vuosittain ja sukupuolittain
            
            Datan lähde [Stat.fi](https://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__vrm__muutl/statfin_muutl_pxt_11a4.px/).
        
            '''),
        html.Iframe(src="https://app.powerbi.com/view?r=eyJrIjoiYTY2NGE0N2MtMDQ3Yi00ZjZhLWE3ODItYThhNTJjYjU0NzY0IiwidCI6ImM4NTBmZTljLWI0NmMtNGIyZC1iODYzLTAxZmEyYTg5ODA2OCIsImMiOjh9",
                style={"height": "1067px", "width": "100%"})])
])

 


@app.callback(Output('graph','figure'), 
              Input('Ikä', 'value'))

def display_selected_year(Ikä):
    mask = df["Ikä"] == Ikä
    fig = px.scatter(df[mask], x="Vuosi", y="Väestö", color="Alue",symbol="Sukupuoli",log_x=True,
                     labels={"Sukupuoli": "Sukupuoli"},width=1700, height=700, hover_name="Ikä")
    fig.update_traces(marker=dict(size=12,
                              line=dict(width=2)),
                  selector=dict(mode='markers'))
    fig.update_traces(mode="markers")
    return fig

    
                    
@app.callback(Output('graph2','figure'), 
              Input('slider', 'value'))

def update_figure(selected_year):
    filtered_df = df2[df2.Vuosi == selected_year]

    fig=px.choropleth(filtered_df,width=1700, height=700, geojson=geojson, color="pcterotus", 
                    locations="Alue", featureidkey="properties.Kunta",color_continuous_scale="Viridis",
                    projection="mercator", facet_col="Sukupuoli", title="Sukupuoli",
                    labels=dict(pcterotus="Vuosittainen muutos", Alue="Maakunta",range_color=[-5, 5]))
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(hovermode="x unified")
    


    return fig

@app.callback(
    Output("graph3", "figure"), 
    [Input("dropdown", "value")])

def update_bar_chart(Alue):
    mask = df3["Alue"] == Alue
    fig = px.bar(df3[mask], x="Alue", y="Väestö", barmode="group", facet_col="Sukupuoli",
                 color="Ikä", height=700,
                 animation_frame="Vuosi", animation_group="Alue",
             category_orders={"Ikä": ["Thur", "Fri", "Sat", "Sun"],
                              "Sukupuoli": ["Lunch", "Dinner"]})
    fig.update_traces(marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)

    return fig



@app.callback(
    Output("graph4", "figure"), 
    [Input("dropdown", "value")])

def update_bar_chart(Alue):
    mask = df4["Alue"] == Alue
    df = px.data.stocks(indexed=True)-1
    fig = px.area(df4[mask], x="Vuosi", y="Yhteensäerotus", height=700, hover_name="henkilöt", color="sukupuoli")
    
    
    
    return fig

app.run_server(port=8059)